In [1]:
import pandas as pd
import datetime as dt

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
path = "/content/gdrive/MyDrive/ML-Transportation-Rate/Raw_Data_Past_Four_Month"
MT_data_ori = pd.read_csv(path +'/Merged_DataFrame_hash_Version_addition.csv')
DAT_data = pd.read_excel(path +'/DAT_Rates_from_DB-9-21.xlsx')

In [14]:
MT_data = MT_data_ori.sample(10000)

# Create a column called ID_OR_OPTIONAL_FIELD, which join first three letter of ORIGIN zip
# and the first three letter of DEST zip and connect them by using "_"
MT_data['ID_OR_OPTIONAL_FIELD'] = MT_data['ORIGIN ZIP'].astype(str).str[:3] + '_' + MT_data['DEST ZIP'].astype(str).str[:3] 
MT_data['ID_OR_OPTIONAL_FIELD'] = MT_data['ID_OR_OPTIONAL_FIELD'].astype(str)


#Remove Columns of 'Unnamed: 0.1','Unnamed: 0'
MT_data = MT_data.drop(['Unnamed: 0.1','Unnamed: 0'], axis = 1)

# Drop the row if na is found in the MT_data['DISTANCE']
MT_data = MT_data[pd.notnull(MT_data['DISTANCE'])]

# Drop the row if na is found in the MT_data['WEIGHT]
MT_data = MT_data[pd.notnull(MT_data['WEIGHT'])]

# Drop the row if na is found in the MT_data['VOLUME']
MT_data = MT_data[pd.notnull(MT_data['VOLUME'])]


# Drop the row if na is found in the MT_data['ORIGIN CITY']
MT_data = MT_data[pd.notnull(MT_data['ORIGIN CITY'])]

# Only conisder shipment has Original zip code in USA and Canada
# Drop the row if ns is found in the MT_data['ORIGIN STATE']
MT_data = MT_data[pd.notnull(MT_data['ORIGIN STATE'])]
MT_data = MT_data.reset_index(drop=True)

US_list = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']
Canada_list = ['AB','BC','MB','NB','NL','NT','NS','NU','ON','PE','QC','SK','YT']
State_list = US_list + Canada_list

for i in range(len(MT_data)):
    if MT_data.loc[i,'ORIGIN STATE'] in State_list:
        pass
    else:
        MT_data = MT_data.drop(i)
        
        
# Drop the row if na is found in the MT_data['ORIGIN ZIP']
MT_data = MT_data[pd.notnull(MT_data['ORIGIN ZIP'])]
MT_data = MT_data.reset_index(drop=True)
# Change 4 digits zips to 5 digis
for i in (range(len(MT_data['ORIGIN ZIP']))):
    zipCode = str(MT_data.loc[i,'ORIGIN ZIP'])
    if zipCode.isdigit():
        if len(zipCode) < 5:
            MT_data.loc[i,'ORIGIN ZIP'] = "0" * (5 - len(zipCode)) + zipCode
    else:
        pass
            


# Actual mode will only consider percel, LTL, and TL
# Drop the row if ns is found in the MT_data['ACTUAL MODE']
MT_data = MT_data[pd.notnull(MT_data['ACTUAL MODE'])]
MT_data['ACTUAL MODE'] = MT_data['ACTUAL MODE'].str.split('.').str[-1] 
MT_data['ACTUAL MODE'] = MT_data['ACTUAL MODE'].str.upper()
MT_data = MT_data.reset_index(drop=True)

acutual_mode_list = ['LTL','TL','PARCEL','INTERMODAL']

for i in range(len(MT_data)):
    if MT_data.loc[i,'ACTUAL MODE'] in acutual_mode_list:
        pass
    else:
        MT_data = MT_data.drop(i)
        
# Remove any nan cell in ACTUAL EQUIP
# Tokenize part information from the original cells
MT_data = MT_data[pd.notnull(MT_data['ACTUAL EQUIP'])]
MT_data['ACTUAL EQUIP'] = MT_data['ACTUAL EQUIP'].str.split('.').str[-1]

        
# Remove any nan cell in LINEHAUL COSTS
MT_data = MT_data[pd.notnull(MT_data['LINEHAUL COSTS'])]

# Remove any nan cell in FUEL COSTS
MT_data = MT_data[pd.notnull(MT_data['FUEL COSTS'])]

# Remove any nan cell in ACC. COSTS
MT_data = MT_data[pd.notnull(MT_data['ACC. COSTS'])]

# Remove any nan cell in total ACTUAL COST
MT_data = MT_data[pd.notnull(MT_data['TOTAL ACTUAL COST'])]

# Add a LINEHAUL UNIT COSTS to the dataframe
MT_data['LINEHAUL UNIT COSTS'] = MT_data['LINEHAUL COSTS']/MT_data['DISTANCE']

# Drop the row if na is found in the MT_data['DEST CITY']
MT_data = MT_data[pd.notnull(MT_data['DEST CITY'])]

# Only conisder shipment has dest zip code in USA and Canada
# Drop the row if ns is found in the MT_data['ORIGIN STATE']
MT_data = MT_data[pd.notnull(MT_data['DEST STATE'])]
MT_data = MT_data.reset_index(drop=True)

for i in range(len(MT_data)):
    if MT_data.loc[i,'DEST STATE'] in State_list:
        pass
    else:
        MT_data = MT_data.drop(i)
        
# Drop the row if na is found in the MT_data['DEST ZIP']
MT_data = MT_data[pd.notnull(MT_data['DEST ZIP'])]
MT_data = MT_data.reset_index(drop=True)
# Change 4 digits zips to 5 digis
for i in (range(len(MT_data['DEST ZIP']))):
    zipCode = str(MT_data.loc[i,'DEST ZIP'])
    if zipCode.isdigit():
        if len(zipCode) < 5:
            MT_data.loc[i,'DEST ZIP'] = "0" * (5 - len(zipCode)) + zipCode
    else:
        pass

# Clean the column of PU_APPT as a new column called File_paried_data
MT_data['File_paried_data'] = pd.to_datetime(MT_data['PU_APPT'].str.split(' ').str[0], errors = 'coerce')
MT_data['File_paried_data'] = MT_data['File_paried_data'].dt.strftime('%Y-%m-%d')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [11]:
#DATA_data Cleaning

# Set ID_OR_OPTIONAL_FIELD as an objective column
DAT_data['ID_OR_OPTIONAL_FIELD'] = DAT_data['ID_OR_OPTIONAL_FIELD'].astype(str)

# Clean the column of File_data  and add year '2022' before the month
DAT_data['File_data'] = DAT_data['SOURCE_FILE_NAME'].str.split(' ').str[-1]
DAT_data['File_data'] = DAT_data['File_data'].str.split('.').str[0]
DAT_data['File_data'] = '2022-' + DAT_data['File_data']
DAT_data['File_data'] = pd.to_datetime(DAT_data['File_data'])

# Remove columns have ALL na
DAT_data = DAT_data.drop('NOTE',axis = 1)


# Drop the row if na is found in the DAT_data['SPOT_AVG_LINEHAUL_RATE']
DAT_data = DAT_data[pd.notnull(DAT_data['SPOT_AVG_LINEHAUL_RATE'])]

# Drop the row if na is found in the DAT_data['SPOT_TIME_FRAME']
DAT_data = DAT_data[pd.notnull(DAT_data['SPOT_TIME_FRAME'])]

# Drop the row if na is found in the DAT_data['CONTRACT_AVG_LINEHAUL_RATE']
DAT_data = DAT_data[pd.notnull(DAT_data['CONTRACT_AVG_LINEHAUL_RATE'])]

# Drop the row if na is found in the DAT_data['CONTRACT_TIME_FRAME']
DAT_data = DAT_data[pd.notnull(DAT_data['CONTRACT_TIME_FRAME'])]

In [15]:
# Join two tables
#Join the table by setting ID_OR_OPTIONAL_FIELD as the key
merge_df = MT_data.merge(DAT_data, on = 'ID_OR_OPTIONAL_FIELD', how = 'left')

In [16]:
# Data Cleaning
# 1. Remove if File_data is NA
merge_df_remove_NA = merge_df
merge_df_remove_NA = merge_df_remove_NA[pd.notnull(merge_df_remove_NA['File_data'])] 
merge_df_remove_NA = merge_df_remove_NA.reset_index(drop=True)


# 2. Only keeps row if File_paried_data is not larger than File_data
#for i in range(len(merge_df)):
#    if pd.to_datetime(merge_df.loc[i, 'File_paried_data']) <= pd.to_datetime(merge_df.loc[i, 'File_data']):
#        merge_df = merge_df.drop(index=i)

for i in range(len(merge_df_remove_NA)):
    if pd.to_datetime(merge_df_remove_NA.loc[i, 'File_paried_data']) <= pd.to_datetime(merge_df_remove_NA.loc[i, 'File_data']):
        merge_df_remove_NA = merge_df_remove_NA.drop(index=i)


In [17]:
#Show all columns in the file
pd.set_option('display.max_columns', None)
merge_df.head()

,SHIPMENT ID,CUSTOMER,DISTANCE,CASES,WEIGHT,VOLUME,SOURCE LOCATION ID,ORIGIN NAME,ORIGIN CITY,ORIGIN STATE,ORIGIN ZIP,DEST LOCATION ID,CONSIGNEE NAME,DEST CITY,DEST STATE,DEST ZIP,ACTUAL CARRIER,ACTUAL MODE,ACTUAL EQUIP,LINEHAUL COSTS,FUEL COSTS,ACC. COSTS,TOTAL ACTUAL COST,PU_APPT,DL_APPT,PU_ARRIVAL (X3),PU_DEPARTED (AF),DL_ARRIVAL (X1),DL_DEPARTED (D1),Insert Date,ID_OR_OPTIONAL_FIELD,LINEHAUL UNIT COSTS,File_paried_data,PC_MILER_PRACTICAL_MILEAGE,SPOT_AVG_LINEHAUL_RATE,SPOT_LOW_LINEHAUL_RATE,SPOT_HIGH_LINEHAUL_RATE,SPOT_FUEL_SURCHARGE,SPOT_TIME_FRAME,SPOT_ORIGIN_GEO_EXPANSION,SPOT_DESTINATION_GEO_EXPANSION,SPOT_NUMBER_OF_COMPANIES,SPOT_NUMBER_OF_REPORTS,SPOT_LINEHAUL_RATE_STDDEV,SPOT_YOUR_OWN_AVG_LINEHAUL_RATE,SPOT_YOUR_OWN_NUMBER_OF_REPORTS,SPOT_ERROR,CONTRACT_AVG_LINEHAUL_RATE,CONTRACT_LOW_LINEHAUL_RATE,CONTRACT_HIGH_LINEHAUL_RATE,CONTRACT_FUEL_SURCHARGE,CONTRACT_AVG_ACCESSORIAL_EXCLUDES_FUEL,CONTRACT_TIME_FRAME,CONTRACT_ORIGIN_GEO_EXPANSION,CONTRACT_DESTINATION_GEO_EXPANSION,CONTRACT_NUMBER_OF_COMPANIES,CONTRACT_NUMBER_OF_REPORTS,CONTRACT_LINEHAUL_RATE_STDDEV,CONTRACT_YOUR_OWN_AVG_LINEHAUL_RATE,CONTRACT_YOUR_OWN_NUMBER_OF_REPORTS,CONTRACT_ERROR,ORIG_CITY,ORIG_STATE,ORIG_POSTAL_CODE,DEST_CITY,DEST_STATE,DEST_POSTAL_CODE,TRUCK_TYPE,RUN_ID,SOURCE_FILE_NAME,DWH_INSERT_DATE,File_data
0,EXEL.B21306E08821,562003601178532658,495.0,1,0.63,0.0,-7414928951098249859,6249500791195756829,HORN LAKE,MS,38637,-8506997517281665882,1969088168317837518,CINCINNATI,OH,45223,1896630384904438820,PARCEL,53DV,9.74,0.0,0.0,9.74,11/3/2021 22:11,11/4/2021 12:11,NO EVENT,NO EVENT,NO EVENT,NO EVENT,09:52.0,386_452,0.019677,2021-11-03,498.0,2.85,2.44,3.36,0.70,3.0,20-Mkt area,20-Mkt area,11.0,26.0,0.57,NaN,NaN,NaN,3.61,3.28,4.32,0.70,156.75,30.0,10-3-Digit Zip,20-Mkt area,5.0,83.0,0.54,NaN,NaN,NaN,NaN,MS,38637,NaN,OH,45223,V,dfa3af1b-8fa1-468a-8ad5-dc6a011af432,CI Rate 7-20.csv,2022-09-21 14:47:34.003,2022-07-20
1,EXEL.B21306E08821,562003601178532658,495.0,1,0.63,0.0,-7414928951098249859,6249500791195756829,HORN LAKE,MS,38637,-8506997517281665882,1969088168317837518,CINCINNATI,OH,45223,1896630384904438820,PARCEL,53DV,9.74,0.0,0.0,9.74,11/3/2021 22:11,11/4/2021 12:11,NO EVENT,NO EVENT,NO EVENT,NO EVENT,09:52.0,386_452,0.019677,2021-11-03,498.0,2.88,2.53,3.21,0.73,7.0,10-3-Digit Zip,20-Mkt area,10.0,18.0,0.76,NaN,NaN,NaN,3.54,3.30,4.02,0.73,158.56,30.0,10-3-Digit Zip,20-Mkt area,5.0,52.0,0.50,NaN,NaN,NaN,NaN,MS,38637,NaN,OH,45223,V,660e1b5d-02b4-4939-9bde-d8cd9608c5a8,CI Rates 5-12.csv,2022-09-21 14:47:40.700,2022-05-12
2,EXEL.B21306E08821,562003601178532658,495.0,1,0.63,0.0,-7414928951098249859,6249500791195756829,HORN LAKE,MS,38637,-8506997517281665882,1969088168317837518,CINCINNATI,OH,45223,1896630384904438820,PARCEL,53DV,9.74,0.0,0.0,9.74,11/3/2021 22:11,11/4/2021 12:11,NO EVENT,NO EVENT,NO EVENT,NO EVENT,09:52.0,386_452,0.019677,2021-11-03,498.0,2.67,2.25,3.06,0.74,7.0,10-3-Digit Zip,20-Mkt area,10.0,14.0,0.55,NaN,NaN,NaN,3.11,2.96,3.63,0.74,158.02,30.0,10-3-Digit Zip,20-Mkt area,3.0,41.0,0.50,NaN,NaN,NaN,NaN,MS,38637,NaN,OH,45223,V,6fad52d5-d4bd-4c08-9cc1-2883ff4ddf92,CI Rate 6-15.csv,2022-09-21 14:46:58.917,2022-06-15
3,EXEL.B21306E08821,562003601178532658,495.0,1,0.63,0.0,-7414928951098249859,6249500791195756829,HORN LAKE,MS,38637,-8506997517281665882,1969088168317837518,CINCINNATI,OH,45223,1896630384904438820,PARCEL,53DV,9.74,0.0,0.0,9.74,11/3/2021 22:11,11/4/2021 12:11,NO EVENT,NO EVENT,NO EVENT,NO EVENT,09:52.0,386_452,0.019677,2021-11-03,512.0,2.67,2.25,3.06,0.74,7.0,10-3-Digit Zip,20-Mkt area,10.0,14.0,0.55,NaN,NaN,NaN,3.11,2.96,3.63,0.74,158.02,30.0,10-3-Digit Zip,20-Mkt area,3.0,41.0,0.50,NaN,NaN,NaN,NaN,MS,38611,NaN,OH,45241,V,6fad52d5-d4bd-4c08-9cc1-2883ff4ddf92,CI Rate 6-15.csv,2022-09-21 14:46:58.917,2022-06-15
4,EXEL.B21306E08821,562003601178532658,495.0,1,0.63,0.0,-7414928951098249859,6249500791195756829,HORN LAKE,MS,38637,-8506997517281665882,1969088168317837518,CINCINNATI,OH,45223,1896630384904438820,PARCEL,53DV,9.74,0.0,0.0,9.74,1

In [18]:
merge_df_remove_NA.head()

,SHIPMENT ID,CUSTOMER,DISTANCE,CASES,WEIGHT,VOLUME,SOURCE LOCATION ID,ORIGIN NAME,ORIGIN CITY,ORIGIN STATE,ORIGIN ZIP,DEST LOCATION ID,CONSIGNEE NAME,DEST CITY,DEST STATE,DEST ZIP,ACTUAL CARRIER,ACTUAL MODE,ACTUAL EQUIP,LINEHAUL COSTS,FUEL COSTS,ACC. COSTS,TOTAL ACTUAL COST,PU_APPT,DL_APPT,PU_ARRIVAL (X3),PU_DEPARTED (AF),DL_ARRIVAL (X1),DL_DEPARTED (D1),Insert Date,ID_OR_OPTIONAL_FIELD,LINEHAUL UNIT COSTS,File_paried_data,PC_MILER_PRACTICAL_MILEAGE,SPOT_AVG_LINEHAUL_RATE,SPOT_LOW_LINEHAUL_RATE,SPOT_HIGH_LINEHAUL_RATE,SPOT_FUEL_SURCHARGE,SPOT_TIME_FRAME,SPOT_ORIGIN_GEO_EXPANSION,SPOT_DESTINATION_GEO_EXPANSION,SPOT_NUMBER_OF_COMPANIES,SPOT_NUMBER_OF_REPORTS,SPOT_LINEHAUL_RATE_STDDEV,SPOT_YOUR_OWN_AVG_LINEHAUL_RATE,SPOT_YOUR_OWN_NUMBER_OF_REPORTS,SPOT_ERROR,CONTRACT_AVG_LINEHAUL_RATE,CONTRACT_LOW_LINEHAUL_RATE,CONTRACT_HIGH_LINEHAUL_RATE,CONTRACT_FUEL_SURCHARGE,CONTRACT_AVG_ACCESSORIAL_EXCLUDES_FUEL,CONTRACT_TIME_FRAME,CONTRACT_ORIGIN_GEO_EXPANSION,CONTRACT_DESTINATION_GEO_EXPANSION,CONTRACT_NUMBER_OF_COMPANIES,CONTRACT_NUMBER_OF_REPORTS,CONTRACT_LINEHAUL_RATE_STDDEV,CONTRACT_YOUR_OWN_AVG_LINEHAUL_RATE,CONTRACT_YOUR_OWN_NUMBER_OF_REPORTS,CONTRACT_ERROR,ORIG_CITY,ORIG_STATE,ORIG_POSTAL_CODE,DEST_CITY,DEST_STATE,DEST_POSTAL_CODE,TRUCK_TYPE,RUN_ID,SOURCE_FILE_NAME,DWH_INSERT_DATE,File_data
40,EXEL.B22136E06598,2941159810065373912,1036.50,4,0.59,0.01,-6304456864783580834,-939722273477065836,Pleasant Prairie,WI,53158,-7514740753024571005,908941813343451349,DENVER,CO,80238,-4290289884386242506,PARCEL,53DV,4.07,0.0,0.0,4.07,5/17/2022 20:05,5/20/2022 14:05,5/17/2022 15:35,5/17/2022 0:00,NO EVENT,5/20/2022 16:07,56:36.0,531_802,0.003927,2022-05-17,1034.0,2.58,2.30,2.85,0.73,7.0,20-Mkt area,20-Mkt area,11.0,15.0,0.49,NaN,NaN,NaN,3.19,2.62,3.50,0.73,158.56,30.0,20-Mkt area,20-Mkt area,9.0,88.0,0.51,NaN,NaN,NaN,NaN,WI,53158,NaN,CO,80238,V,660e1b5d-02b4-4939-9bde-d8cd9608c5a8,CI Rates 5-12.csv,2022-09-21 14:47:40.700,2022-05-12
48,EXEL.B22138E08960,562003601178532658,1353.57,1,6.00,700.00,-7414928951098249859,6249500791195756829,HORN LAKE,MS,38637,7107389593678065827,-4328145684075287046,BOSTON,MA,02118-3522,-4290289884386242506,PARCEL,53DV,5.72,0.0,0.0,5.72,5/20/2022 22:05,5/25/2022 19:05,NO EVENT,NO EVENT,NO EVENT,NO EVENT,29:24.0,386_021,0.004226,2022-05-20,1342.0,2.26,2.07,2.46,0.73,7.0,20-Mkt area,20-Mkt area,10.0,17.0,0.29,NaN,NaN,NaN,3.07,2.70,3.35,0.73,158.56,30.0,20-Mkt area,20-Mkt area,9.0,21.0,0.30,NaN,NaN,NaN,NaN,MS,38611,NaN,MA,2148,V,660e1b5d-02b4-4939-9bde-d8cd9608c5a8,CI Rates 5-12.csv,2022-09-21 14:47:40.700,2022-05-12
85,MTNORAM.B22152M03498,7485008617214758133,342.40,1,0.36,0.02,-8042456250106848909,-1774819625051906144,KNOXVILLE,TN,37921,1379375821000977788,2353432571711931620,SIDNEY,OH,45365-1735,2674631463645570409,PARCEL,53DV,0.36,0.0,0.0,0.36,6/1/2022 21:06,6/2/2022 17:06,6/1/2022 17:32,6/1/2022 23:27,NO EVENT,6/2/2022 13:59,03:28.0,379_453,0.001051,2022-06-01,319.0,2.46,2.12,2.83,0.73,3.0,20-Mkt area,20-Mkt area,15.0,23.0,0.54,NaN,NaN,NaN,3.04,2.01,3.52,0.73,158.56,30.0,10-3-Digit Zip,20-Mkt area,7.0,46.0,0.71,NaN,NaN,NaN,NaN,TN,37922,NaN,OH,45371,V,660e1b5d-02b4-4939-9bde-d8cd9608c5a8,CI Rates 5-12.csv,2022-09-21 14:47:40.700,2022-05-12
93,MTNORAM.B22250M03372,7485008617214758133,313.10,1,0.86,0.07,-8042456250106848909,-1774819625051906144,KNOXVILLE,TN,37921,8633495325340387974,6757351254457610458,NEW LEBANON,OH,45345-1254,2674631463645570409,PARCEL,53DV,0.86,0.0,0.0,0.86,9/12/2022 21:09,9/13/2022 18:09,9/12/2022 17:50,9/13/2022 0:41,NO EVENT,9/13/2022 14:24,06:53.0,379_453,0.002747,2022-09-12,319.0,2.45,2.29,2.65,0.70,3.0,20-Mkt area,20-Mkt area,7.0,10.0,0.30,NaN,NaN,NaN,2.94,1.91,3.15,0.70,156.75,30.0,10-3-Digit Zip,20-Mkt area,7.0,46.0,0.69,NaN,NaN,NaN,NaN,TN,37922,NaN,OH,45371,V,dfa3af1b-8fa1-468a-8ad5-dc6a011af432,CI Rate 7-20.csv,2022-09-21 14:47:34.003,2022-07-20
94,MTNORAM.B22250M03372,7485008617214758133,313.10,1,0.86,0.07,-8042456250106848909,-1774819625051906144,KNOXVILLE,TN,37921,8633495325340387974,6757

In [19]:
# Review column name
merge_df_remove_NA.columns

Index(['SHIPMENT ID', 'CUSTOMER', 'DISTANCE', 'CASES', 'WEIGHT', 'VOLUME',
       'SOURCE LOCATION ID', 'ORIGIN NAME', 'ORIGIN CITY', 'ORIGIN STATE',
       'ORIGIN ZIP', 'DEST LOCATION ID', 'CONSIGNEE NAME', 'DEST CITY',
       'DEST STATE', 'DEST ZIP', 'ACTUAL CARRIER', 'ACTUAL MODE',
       'ACTUAL EQUIP', 'LINEHAUL COSTS', 'FUEL COSTS', 'ACC. COSTS',
       'TOTAL ACTUAL COST', 'PU_APPT', 'DL_APPT', 'PU_ARRIVAL (X3)',
       'PU_DEPARTED (AF)', 'DL_ARRIVAL (X1)', 'DL_DEPARTED (D1)',
       'Insert Date', 'ID_OR_OPTIONAL_FIELD', 'LINEHAUL UNIT COSTS',
       'File_paried_data', 'PC_MILER_PRACTICAL_MILEAGE',
       'SPOT_AVG_LINEHAUL_RATE', 'SPOT_LOW_LINEHAUL_RATE',
       'SPOT_HIGH_LINEHAUL_RATE', 'SPOT_FUEL_SURCHARGE', 'SPOT_TIME_FRAME',
       'SPOT_ORIGIN_GEO_EXPANSION', 'SPOT_DESTINATION_GEO_EXPANSION',
       'SPOT_NUMBER_OF_COMPANIES', 'SPOT_NUMBER_OF_REPORTS',
       'SPOT_LINEHAUL_RATE_STDDEV', 'SPOT_YOUR_OWN_AVG_LINEHAUL_RATE',
       'SPOT_YOUR_OWN_NUMBER_OF_REPORTS', 'SP

In [20]:
#Selected Input columns
required_column = ['SHIPMENT ID', 'CUSTOMER', 'DISTANCE', 'CASES', 'WEIGHT', 'VOLUME',
       'SOURCE LOCATION ID', 'ORIGIN NAME', 'ORIGIN CITY', 'ORIGIN STATE',
       'ORIGIN ZIP', 'DEST LOCATION ID', 'CONSIGNEE NAME', 'DEST CITY',
       'DEST STATE', 'DEST ZIP', 'ACTUAL CARRIER', 'ACTUAL MODE',
       'ACTUAL EQUIP', 'LINEHAUL COSTS','TOTAL ACTUAL COST', 'PU_APPT','DL_APPT',
       'LINEHAUL UNIT COSTS','PC_MILER_PRACTICAL_MILEAGE',
       'SPOT_AVG_LINEHAUL_RATE', 'SPOT_LOW_LINEHAUL_RATE',
       'SPOT_HIGH_LINEHAUL_RATE', 'SPOT_FUEL_SURCHARGE', 'SPOT_TIME_FRAME',
       'SPOT_ORIGIN_GEO_EXPANSION', 'SPOT_DESTINATION_GEO_EXPANSION',
       'SPOT_NUMBER_OF_COMPANIES', 'SPOT_NUMBER_OF_REPORTS',
       'SPOT_LINEHAUL_RATE_STDDEV', 'SPOT_YOUR_OWN_AVG_LINEHAUL_RATE',
       'SPOT_YOUR_OWN_NUMBER_OF_REPORTS', 'SPOT_ERROR',
       'CONTRACT_AVG_LINEHAUL_RATE', 'CONTRACT_LOW_LINEHAUL_RATE',
       'CONTRACT_HIGH_LINEHAUL_RATE', 'CONTRACT_FUEL_SURCHARGE',
       'CONTRACT_AVG_ACCESSORIAL_EXCLUDES_FUEL', 'CONTRACT_TIME_FRAME',
       'CONTRACT_ORIGIN_GEO_EXPANSION', 'CONTRACT_DESTINATION_GEO_EXPANSION',
       'CONTRACT_NUMBER_OF_COMPANIES', 'CONTRACT_NUMBER_OF_REPORTS',
       'CONTRACT_LINEHAUL_RATE_STDDEV', 'CONTRACT_YOUR_OWN_AVG_LINEHAUL_RATE',
       'CONTRACT_YOUR_OWN_NUMBER_OF_REPORTS', 'CONTRACT_ERROR', 'ORIG_CITY',
       'ORIG_STATE', 'ORIG_POSTAL_CODE', 'DEST_CITY', 'DEST_STATE',
       'DEST_POSTAL_CODE', 'TRUCK_TYPE', 'RUN_ID', 'SOURCE_FILE_NAME']
cleaned_df = merge_df_remove_NA[required_column]

In [21]:
cleaned_df.head()

,SHIPMENT ID,CUSTOMER,DISTANCE,CASES,WEIGHT,VOLUME,SOURCE LOCATION ID,ORIGIN NAME,ORIGIN CITY,ORIGIN STATE,ORIGIN ZIP,DEST LOCATION ID,CONSIGNEE NAME,DEST CITY,DEST STATE,DEST ZIP,ACTUAL CARRIER,ACTUAL MODE,ACTUAL EQUIP,LINEHAUL COSTS,TOTAL ACTUAL COST,PU_APPT,DL_APPT,LINEHAUL UNIT COSTS,PC_MILER_PRACTICAL_MILEAGE,SPOT_AVG_LINEHAUL_RATE,SPOT_LOW_LINEHAUL_RATE,SPOT_HIGH_LINEHAUL_RATE,SPOT_FUEL_SURCHARGE,SPOT_TIME_FRAME,SPOT_ORIGIN_GEO_EXPANSION,SPOT_DESTINATION_GEO_EXPANSION,SPOT_NUMBER_OF_COMPANIES,SPOT_NUMBER_OF_REPORTS,SPOT_LINEHAUL_RATE_STDDEV,SPOT_YOUR_OWN_AVG_LINEHAUL_RATE,SPOT_YOUR_OWN_NUMBER_OF_REPORTS,SPOT_ERROR,CONTRACT_AVG_LINEHAUL_RATE,CONTRACT_LOW_LINEHAUL_RATE,CONTRACT_HIGH_LINEHAUL_RATE,CONTRACT_FUEL_SURCHARGE,CONTRACT_AVG_ACCESSORIAL_EXCLUDES_FUEL,CONTRACT_TIME_FRAME,CONTRACT_ORIGIN_GEO_EXPANSION,CONTRACT_DESTINATION_GEO_EXPANSION,CONTRACT_NUMBER_OF_COMPANIES,CONTRACT_NUMBER_OF_REPORTS,CONTRACT_LINEHAUL_RATE_STDDEV,CONTRACT_YOUR_OWN_AVG_LINEHAUL_RATE,CONTRACT_YOUR_OWN_NUMBER_OF_REPORTS,CONTRACT_ERROR,ORIG_CITY,ORIG_STATE,ORIG_POSTAL_CODE,DEST_CITY,DEST_STATE,DEST_POSTAL_CODE,TRUCK_TYPE,RUN_ID,SOURCE_FILE_NAME
40,EXEL.B22136E06598,2941159810065373912,1036.50,4,0.59,0.01,-6304456864783580834,-939722273477065836,Pleasant Prairie,WI,53158,-7514740753024571005,908941813343451349,DENVER,CO,80238,-4290289884386242506,PARCEL,53DV,4.07,4.07,5/17/2022 20:05,5/20/2022 14:05,0.003927,1034.0,2.58,2.30,2.85,0.73,7.0,20-Mkt area,20-Mkt area,11.0,15.0,0.49,NaN,NaN,NaN,3.19,2.62,3.50,0.73,158.56,30.0,20-Mkt area,20-Mkt area,9.0,88.0,0.51,NaN,NaN,NaN,NaN,WI,53158,NaN,CO,80238,V,660e1b5d-02b4-4939-9bde-d8cd9608c5a8,CI Rates 5-12.csv
48,EXEL.B22138E08960,562003601178532658,1353.57,1,6.00,700.00,-7414928951098249859,6249500791195756829,HORN LAKE,MS,38637,7107389593678065827,-4328145684075287046,BOSTON,MA,02118-3522,-4290289884386242506,PARCEL,53DV,5.72,5.72,5/20/2022 22:05,5/25/2022 19:05,0.004226,1342.0,2.26,2.07,2.46,0.73,7.0,20-Mkt area,20-Mkt area,10.0,17.0,0.29,NaN,NaN,NaN,3.07,2.70,3.35,0.73,158.56,30.0,20-Mkt area,20-Mkt area,9.0,21.0,0.30,NaN,NaN,NaN,NaN,MS,38611,NaN,MA,2148,V,660e1b5d-02b4-4939-9bde-d8cd9608c5a8,CI Rates 5-12.csv
85,MTNORAM.B22152M03498,7485008617214758133,342.40,1,0.36,0.02,-8042456250106848909,-1774819625051906144,KNOXVILLE,TN,37921,1379375821000977788,2353432571711931620,SIDNEY,OH,45365-1735,2674631463645570409,PARCEL,53DV,0.36,0.36,6/1/2022 21:06,6/2/2022 17:06,0.001051,319.0,2.46,2.12,2.83,0.73,3.0,20-Mkt area,20-Mkt area,15.0,23.0,0.54,NaN,NaN,NaN,3.04,2.01,3.52,0.73,158.56,30.0,10-3-Digit Zip,20-Mkt area,7.0,46.0,0.71,NaN,NaN,NaN,NaN,TN,37922,NaN,OH,45371,V,660e1b5d-02b4-4939-9bde-d8cd9608c5a8,CI Rates 5-12.csv
93,MTNORAM.B22250M03372,7485008617214758133,313.10,1,0.86,0.07,-8042456250106848909,-1774819625051906144,KNOXVILLE,TN,37921,8633495325340387974,6757351254457610458,NEW LEBANON,OH,45345-1254,2674631463645570409,PARCEL,53DV,0.86,0.86,9/12/2022 21:09,9/13/2022 18:09,0.002747,319.0,2.45,2.29,2.65,0.70,3.0,20-Mkt area,20-Mkt area,7.0,10.0,0.30,NaN,NaN,NaN,2.94,1.91,3.15,0.70,156.75,30.0,10-3-Digit Zip,20-Mkt area,7.0,46.0,0.69,NaN,NaN,NaN,NaN,TN,37922,NaN,OH,45371,V,dfa3af1b-8fa1-468a-8ad5-dc6a011af432,CI Rate 7-20.csv
94,MTNORAM.B22250M03372,7485008617214758133,313.10,1,0.86,0.07,-8042456250106848909,-1774819625051906144,KNOXVILLE,TN,37921,8633495325340387974,6757351254457610458,NEW LEBANON,OH,45345-1254,2674631463645570409,PARCEL,53DV,0.86,0.86,9/12/2022 21:09,9/13/2022 18:09,0.002747,319.0,2.46,2.12,2.83,0.73,3.0,20-Mkt area,20-Mkt area,15.0,23.0,0.54,NaN,NaN,NaN,3.04,2.01,3.52,0.73,158.56,30.0,10-3-Digit Zip,20-Mkt area,7.0,46.0,0.71,NaN,NaN,NaN,NaN,TN,37922,NaN,OH,45371,V,660e1b5d-02b4-4939-9bde-d8cd9608c5a8,CI Rates 5-12.csv


# Output Dataset

In [23]:
Output_path = "/content/gdrive/MyDrive/ML-Transportation-Rate/sample data"
cleaned_df.to_csv( Output_path +'/sample_data.csv')
